## What is Django?

Django (/ˈdʒæŋɡoʊ/ jang-goh) is a free and open source web application framework, written in Python. A web framework is a set of components that helps you to develop websites faster and easier.

### Creating a Django Project Directory

> mkdir project_name
<br>
> cd project_name

### Virtual Environment

> python3 -m venv myvenv

#### Starting a virtual environment

> source myvenv/bin/activate (Linux)
<br><br>
> myvenv\Scripts\activate (Windows)

#### Installing Django

> (myvenv) ~$ python3 -m pip install --upgrade pip

<p>Create requirements.txt file inside the project directory and add Django to it</p>

> (myvenv) ~$ pip install -r requirements.txt

#### Creating a Django Project

> (myvenv) ~/<proj_dir>$ django-admin startproject mysite . (Linux)
<br><br>
> (myvenv) C:\Users\Name\PROJ_DIR> django-admin.exe startproject mysite . (Windows)

<p>Add allowed hosts to manage.py</p>

<p>ALLOWED_HOSTS = ['127.0.0.1', 'localhost']</p>

<p>Create Database</p>

> (myvenv) ~/<proj_dir>$ python settings.py migrate

#### Starting the web server

> (myvenv) ~/<proj_dir>$ python manage.py runserver

#### Creating an application

> (myvenv) ~/<proj_dir>$ python manage.py startapp blog

<p>Add the app to mysite/settings.py to tell Django about the app</p>

<p>INSTALLED_APPS = [<br>
    'django.contrib.admin',<br>
    'django.contrib.auth',<br>
    'django.contrib.contenttypes',<br>
    'django.contrib.sessions',<br>
    'django.contrib.messages',<br>
    'django.contrib.staticfiles',<br>
    'blog',
]</p>



In [ ]:
# Creating our blogs Post model blog/models.py

from django.conf import settings
from django.db import models
from django.utils import timezone


class Post(models.Model):
    author = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE)
    title = models.CharField(max_length=200)
    text = models.TextField()
    created_date = models.DateTimeField(default=timezone.now)
    published_date = models.DateTimeField(blank=True, null=True)

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title

#### Creating tables in our database

> (myvenv) ~/<proj_dir>$ python manage.py makemigrations blog

#### Applying the created migration file to our database

> (myvenv) ~/<proj_dir>$ python manage.py migrate blog

## Django admin

<p>To add, edit and delete the posts we've just modeled, we will use Django admin.</p>

In [ ]:
# blog/admin.py

from django.contrib import admin
from .models import Post

admin.site.register(Post)

#### Creating a superuser for Admin

> (myvenv) ~/<proj_dir>$ python manage.py createsuperuser

<p>Visit http://localhost:8000/admin</p>

## Pushing our project to Github

> git init
<br>
> git add .
<br>
> git commit -m 'first commit'
<br>
> git remote add origin <git_repo_link>
<br>
> git push origin master

## Deploying App on Heroku

## Django URLs



In [ ]:
# mysite/urls.py

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('blog.urls')),
]

## blog.urls

<p>Create urls.py inside blog app and add the below code</p>

In [ ]:
# blog/urls.py

from django.urls import path
from . import views

urlpatterns = [
    path('', views.post_list, name='post_list'),
]

## Django Views

In [ ]:
# blog/views.py

from django.shortcuts import render

# Create your views here.

def post_list(request):
    return render(request, 'blog/post_list.html', {})

<p>Create the following directories: blog --> templates --> blog</p>

<p>Add post_list.html file with some sample HTML markup</p>

<p>Push updates to Github</p>

## Dynamic data in templates



In [ ]:
# blog/views.py
# add this to import Post

from django.shortcuts import render
from django.utils import timezone
from .models import Post

def post_list(request):
    posts = Post.objects.filter(published_date__lte=timezone.now()).order_by('published_date')
    return render(request, 'blog/post_list.html', {'posts': posts})

In [ ]:
# blog/templates/blog/post_list.html

{% for post in posts %}
    <div>
        <p>published: {{ post.published_date }}</p>
        <h2><a href="">{{ post.title }}</a></h2>
        <p>{{ post.text|linebreaksbr }}</p>
    </div>
{% endfor %}

## Template extending

<p>Create blog/templates/blog/base.html</p>

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Hello!!</title>
</head>
<body>
    <p>Hello from Base!!</p>

    {% block content %}
    {% endblock %}
</body>
</html>

In [1]:
# blog/templates/blog/post_list.html

{% extends 'blog/base.html' %}

{% block content %}
	{% for post in posts %}
	    <div>
	        <p>published: {{ post.published_date }}</p>
	        <h2><a href="">{{ post.title }}</a></h2>
	        <p>{{ post.text|linebreaksbr }}</p>
	    </div>
	{% endfor %}
{% endblock %}

SyntaxError: invalid syntax (<ipython-input-1-53f8e8487e89>, line 3)

## Extending applications

<p>Add the URL to anchor tag in post_list</p>

In [ ]:
# blog/templates/blog/post_list.html

<a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a>

In [ ]:
# Add url in blog/urls.py

from django.urls import path
from . import views

urlpatterns = [
    path('', views.post_list, name='post_list'),
    path('post/<int:pk>/', views.post_detail, name='post_detail'),
]

In [ ]:
# Add post_detail view in blog/views.py

from django.shortcuts import render, get_object_or_404

def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    return render(request, 'blog/post_detail.html', {'post': post})

In [ ]:
# Create post_detail template in blog/templates/blog/post_detail.html

{% extends 'blog/base.html' %}

{% block content %}
    <div class="post">
        {% if post.published_date %}
            <div class="date">
                {{ post.published_date }}
            </div>
        {% endif %}
        <h2>{{ post.title }}</h2>
        <p>{{ post.text|linebreaksbr }}</p>
    </div>
{% endblock %}